# Linear Discriminant Analysis for CRC Prediction

Expleriments with linear discriminant analysis as a classifier.

In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.metrics import accuracy_score, balanced_accuracy_score, make_scorer
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.model_selection import cross_val_score

from IPython.display import set_matplotlib_formats
set_matplotlib_formats('pdf')

In [2]:
# Get features
samples = pd.read_csv("samples.csv", index_col=0)
microbes = pd.read_csv("microbes.csv", index_col=0)
metabolites = pd.read_csv("metabolites.csv", index_col=0)
combined_features = pd.concat([microbes, metabolites], axis=1)
# Label vector 
labels = samples.case

## Microbes Data

Split into training and testing.

In [3]:
# We'll use 80% for training, and 20% for testing
X_train, X_test, labels_train, labels_test = train_test_split(
    microbes, labels, test_size=0.2, random_state=42)

We also set up a 5x cross-validation on the training set, in order to tune the parameters of the model:

In [4]:
# Setup 5x cross-validation, and manually tune the parameters for the best accuracy
# The possible parameters are:
#     solver: 'svd', 'eigen', 'lsqr'
#     shrinkage: None, 'auto', 0 < n < 1

lda = LDA(solver='lsqr', shrinkage='auto')
scores = cross_val_score(lda, X_train, labels_train, cv=5,
                         scoring=make_scorer(roc_auc_score))
print("Training AUC Score: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Training AUC Score: 0.59 (+/- 0.14)


The best model used the 'lsqr' solver 'auto' shrinkage.

Predicting on the testing set:

In [6]:
# Train the best model with the entire training set
lda.fit(X_train, labels_train)
microbes_pred = lda.predict(X_test)
balanced_accuracy = balanced_accuracy_score(labels_test, microbes_pred)
accuracy = accuracy_score(labels_test, microbes_pred)
auc = roc_auc_score(y_true=labels_test, y_score=lda.decision_function(X_test))
print("Accuracy_score:", accuracy)
print("Balanced accuracy score:", balanced_accuracy)
print("AUC score:", auc)

Accuracy_score: 0.666666666667
Balanced accuracy score: 0.618421052632
AUC score: 0.677631578947


In [48]:
[i for i, j in rf.predict_proba(X_test)]

[0.90000000000000002,
 0.69999999999999996,
 0.59999999999999998,
 0.69999999999999996,
 0.40000000000000002,
 0.59999999999999998,
 1.0,
 0.5,
 0.80000000000000004,
 0.80000000000000004,
 0.69999999999999996,
 0.90000000000000002,
 1.0,
 0.29999999999999999,
 0.69999999999999996,
 0.90000000000000002,
 0.69999999999999996,
 0.5,
 0.69999999999999996,
 0.69999999999999996,
 0.90000000000000002,
 0.69999999999999996,
 0.59999999999999998,
 0.69999999999999996,
 1.0,
 0.59999999999999998,
 0.90000000000000002]

In [9]:
# Plot an ROC curve
fpr, tpr, _ = roc_curve(y_true=labels_test, y_score=lda.decision_function(X_test))
plt.plot(fpr, tpr, label="AUC = {0:.2f}".format(auc))
plt.plot([0, 1], [0, 1], color='red', lw=1, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
plt.show()

<Figure size 432x288 with 1 Axes>

In [10]:
# Important variables, based on LDA weights
weights = list(enumerate(np.abs(lda.coef_[0])))
sorted_weights = sorted(weights, key=lambda x: x[1], reverse=True)
# Get top 30 variables
top_variables = [i for i, j in sorted_weights][:20]
[microbes.columns[i] for i in top_variables]

['Root;k__Bacteria;p__Chloroflexi',
 'Root;k__Bacteria;p__Proteobacteria;c__Gammaproteobacteria;o__Alteromonadales',
 'Root;k__Bacteria;p__Bacteroidetes;c__Bacteroidia;o__Bacteroidales;f__[Odoribacteraceae];g__Butyricimonas',
 'Root;k__Bacteria;p__Cyanobacteria',
 'Root;k__Bacteria;p__Fusobacteria;c__Fusobacteria;o__Fusobacteriales;f__Fusobacteriaceae;g__Fusobacterium',
 'Root;k__Bacteria;p__Firmicutes;c__Clostridia;o__Clostridiales;f__Lachnospiraceae;g__Pseudobutyrivibrio',
 'Root;k__Bacteria;p__Firmicutes;c__Bacilli;o__Bacillales;f__Planococcaceae',
 'Root;k__Bacteria;p__Firmicutes;c__Bacilli;o__Bacillales;f__Planococcaceae;g__',
 'Root;k__Bacteria;p__Synergistetes;c__Synergistia;o__Synergistales;f__Synergistaceae',
 'Root;k__Bacteria;p__Synergistetes;c__Synergistia',
 'Root;k__Bacteria;p__Synergistetes;c__Synergistia;o__Synergistales',
 'Root;k__Bacteria;p__Synergistetes',
 'Root;k__Bacteria;p__Firmicutes;c__Clostridia;o__Clostridiales;f__Ruminococcaceae;g__Anaerotruncus',
 'Root;k_

## Metabolites Data

In [11]:
X_train, X_test, labels_train, labels_test = train_test_split(
    metabolites, labels, test_size=0.2, random_state=42)

In [12]:
# Setup 5x cross-validation, and manually tune the parameters for the best accuracy
# The possible parameters are:
#     solver: 'svd', 'eigen', 'lsqr'
#     shrinkage: None, 'auto', 0 < n < 1

lda = LDA(solver='lsqr', shrinkage='auto')
scores = cross_val_score(lda, X_train, labels_train, cv=5,
                         scoring=make_scorer(roc_auc_score))
print("Training AUC Score: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Training AUC Score: 0.69 (+/- 0.12)


In [13]:
# Train the best model with the entire training set
lda.fit(X_train, labels_train)
metab_pred = lda.predict(X_test)
balanced_accuracy = balanced_accuracy_score(labels_test, metab_pred)
accuracy = accuracy_score(labels_test, metab_pred)
auc = roc_auc_score(y_true=labels_test, y_score=lda.decision_function(X_test))
print("Accuracy_score:", accuracy)
print("Balanced accuracy score:", balanced_accuracy)
print("AUC score:", auc)

Accuracy_score: 0.814814814815
Balanced accuracy score: 0.759868421053
AUC score: 0.861842105263


In [14]:
# Important variables, based on LDA weights
weights = list(enumerate(np.abs(lda.coef_[0])))
sorted_weights = sorted(weights, key=lambda x: x[1], reverse=True)
# Get top 30 variables
top_variables = [i for i, j in sorted_weights][:20]
[metabolites.columns[i] for i in top_variables]

['X_15528',
 'ALLO_THREONINE',
 'HEXADECANEDIOATE',
 'N_ACETYLGLUTAMINE',
 'LEVULINATE__4_OXOVALERATE_',
 '_4_HYDROXYBENZOATE',
 'X_14473',
 'PROLINE',
 'ISOPALMITIC_ACID',
 'N6_ACETYLLYSINE',
 'MANNOSE',
 '_4_ACETAMIDOPHENOL',
 'PROLYLALANINE',
 'PALMITOYL_SPHINGOMYELIN',
 '_2_HYDROXYMYRISTATE',
 'HOMOCYSTEINE',
 'GLUTAMINE',
 'N_ACETYLNEURAMINATE',
 'LEUCYLTRYPTOPHAN',
 'HISTIDINE']

In [15]:
# Plot an ROC curve
fpr, tpr, _ = roc_curve(y_true=labels_test, y_score=lda.decision_function(X_test))
plt.plot(fpr, tpr, label="AUC = {0:.2f}".format(auc))
plt.plot([0, 1], [0, 1], color='red', lw=1, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
plt.show()

<Figure size 432x288 with 1 Axes>

## Combined Data

In [16]:
X_train, X_test, labels_train, labels_test = train_test_split(
    combined_features, labels, test_size=0.2, random_state=42)

In [17]:
# Setup 5x cross-validation, and manually tune the parameters for the best accuracy
# The possible parameters are:
#     solver: 'svd', 'eigen', 'lsqr'
#     shrinkage: None, 'auto', 0 < n < 1

lda = LDA(solver='lsqr', shrinkage=.7)
scores = cross_val_score(lda, X_train, labels_train, cv=5,
                         scoring=make_scorer(roc_auc_score))
print("Training AUC Score: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Training AUC Score: 0.65 (+/- 0.10)


In [18]:
# Train the best model with the entire training set
lda.fit(X_train, labels_train)
combined_pred = lda.predict(X_test)
balanced_accuracy = balanced_accuracy_score(labels_test, combined_pred)
accuracy = accuracy_score(labels_test, combined_pred)
auc = roc_auc_score(y_true=labels_test, y_score=lda.decision_function(X_test))
print("Accuracy_score:", accuracy)
print("Balanced accuracy score:", balanced_accuracy)
print("AUC score:", auc)

Accuracy_score: 0.814814814815
Balanced accuracy score: 0.723684210526
AUC score: 0.848684210526


In [19]:
# Plot an ROC curve
fpr, tpr, _ = roc_curve(y_true=labels_test, y_score=lda.decision_function(X_test))
plt.plot(fpr, tpr, label="AUC = {0:.2f}".format(auc))
plt.plot([0, 1], [0, 1], color='red', lw=1, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
plt.show()

<Figure size 432x288 with 1 Axes>

Overall, the performance of the metabolites data alone was the best, with an AUC score of 0.86. Combining the two datasets saw a decrease in performance. It is possible that performing some variable selection to eliminate unimportant variables would improve the model performance.